In [1]:
import pandas as pd
from os import path
pd.options.mode.chained_assignment = None


def extract_information(years):
    data_frames = []
    for y in years:
        file_name = 'assets/season-' + y + ".csv"
        if (path.exists(file_name)):
            data_frames.append(pd.read_csv(file_name))

    # csv = pd.read_csv(file_name,usecols=col_list,index_col="Date")
    csv = pd.concat(data_frames,sort=True).reset_index()
    winning_information(csv)
    final = pd.DataFrame(csv, columns=["Date", "HomeTeam", "AwayTeam", "HS", "AS", "HST", "AST","FTHG","FTAG","WRH",
                                       "WRA", "PH", "PA", "FTR"])
    final.to_csv("pre.csv", index=False)


def winning_information(csv):
    all_team = pd.DataFrame(csv).HomeTeam.unique()  # ottieni tutti i team
    csv.insert(23, 'WRH', 0)
    csv.insert(24, 'WRA', 0)
    csv.insert(25, 'PH', 0)
    csv.insert(26, 'PA', 0)

    for t in all_team:
        result = csv.loc[(csv["HomeTeam"] == t) | (csv["AwayTeam"] == t)]
        add_performance_for_team(csv, result.index, t)




def add_performance_for_team(csv, indexs, team):
    sum = {"res": [],
           "matchs": 0,
           "wr": []}

    for index in indexs:
        update_rate(sum, csv["FTR"][index], csv["HomeTeam"][index] == team)
        add_information_on_csv(csv, index, sum, csv["HomeTeam"][index] == team)

    #print("Performance " + team + "- ")
    #print(sum)


def update_rate(sum, result, isHome):
    if (isHome):
        if (result == 'H'):
            sum["res"].append(1)
        elif (result == 'A'):
            sum["res"].append(-1)
        else:
            sum["res"].append(0)
    else:
        if (result == 'A'):
            sum['res'].append(1)
        elif (result == 'H'):
            sum['res'].append(-1)
        else:
            sum['res'].append(0)

    sum["matchs"] += 1
    sum["wr"].append(round((sum['res'].count(1) * 100 / sum["matchs"]), 1))


def add_information_on_csv(csv, index, data, isHome):
    if (isHome):
        csv["WRH"][index] = data["wr"][-1]
        csv["PH"][index] = calculate_performance_of_last_five(data["res"])

    else:
        csv["WRA"][index] = data["wr"][-1]
        csv["PA"][index] = calculate_performance_of_last_five(data["res"])


def calculate_performance_of_last_five(data):
    if not data[-5:]:
        return 0  # suppongo se non ci sono dati che è stata pareggiata
    else:
        return sum(data[-5:], 0)

In [2]:
extract_information(['1415', '1516', '1617', '1718', '1819'])